In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

def read_titanic_data(url):
  df = pd.read_csv(url)
  y = df["Survived"]
  X = df.drop(["Survived", "Name"], axis = 1)
  return df, X, y

In [7]:
train_url = "https://raw.githubusercontent.com/middlebury-csci-0451/CSCI-0451/main/data/titanic/train.csv"

df_train, X_train, y_train = read_titanic_data(train_url)

In [8]:
X_train = pd.get_dummies(X_train, columns = ["Sex"], drop_first = "if_binary")

In [26]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

def poly_LR(deg):
  return Pipeline([("poly", PolynomialFeatures(degree = deg)),
                   ("LR", LogisticRegression(penalty = "none", max_iter = 1000))])

In [27]:
plr = poly_LR(deg = 2)
plr.fit(X_train, y_train)

/Users/ceceziegler/opt/anaconda3/envs/ml-0451/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/ceceziegler/opt/anaconda3/envs/ml-0451/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('poly', PolynomialFeatures()),
                ('LR', LogisticRegression(max_iter=1000, penalty='none'))])

In [28]:
test_url = "https://raw.githubusercontent.com/middlebury-csci-0451/CSCI-0451/main/data/titanic/test.csv"

df_test, X_test, y_test = read_titanic_data(test_url)
X_test = pd.get_dummies(X_test, columns = ["Sex"], drop_first="if_binary")

In [32]:
y_pred = plr.predict(X_test)

In [33]:
import numpy as np

def fair_pred(y_pred, y_test):
    tp = sum((y_test == 1) & (y_pred == 1))
    tn = sum((y_test == 0) & (y_pred == 0))
    fp = sum((y_test == 0) & (y_pred == 1))
    fn = sum((y_test == 1) & (y_pred == 0))
    
    FNR = fn / (fn + tp)
    FPR = fp / (fp + tn)
    p = (tp + fn) / len(y_test)
    PPV = tp / (tp + fp)
    
    LHS = FPR
    RHS = (p/(1-p)) * ((1-PPV)/PPV) * (1- FNR)
    eq = (LHS, "=", RHS)
    
    return eq

In [35]:
fair_pred(y_pred, y_test)

(0.12280701754385964, '=', 0.12280701754385966)